In [1]:
import numpy as np
from pathlib import Path
import scipy.stats as stats
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from pynwb import NWBHDF5IO # $ pip install -U pynwb


In [2]:
# root folder containing NWB files
nwb_folder_path = Path("/Users/yoon/Downloads/simulation/nwb_files/")
nwb_filepath = nwb_folder_path / f"Apollo_Simulation_thresholded.nwb"

# read NWB file
print(f"Loading nwb file: {nwb_filepath.name}")
io = NWBHDF5IO(nwb_filepath)
nwbfile = io.read()
nwbfile

Loading nwb file: Apollo_Simulation_thresholded.nwb


root pynwb.file.NWBFile at 0x5568536720
Fields:
  analysis: {
    normalizer_analysis_0 <class 'hdmf.common.table.DynamicTable'>
  }
  devices: {
    Device <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    ABC <class 'pynwb.ecephys.ElectrodeGroup'>,
    DEF <class 'pynwb.ecephys.ElectrodeGroup'>,
    GH <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  file_create_date: [datetime.datetime(2024, 12, 23, 17, 11, 58, 735867, tzinfo=tzoffset(None, -18000))]
  identifier: Apollo_Simulation_thresholded
  intervals: {
    trials_table_normalizers_2024-09-24 12:30:21 <class 'pynwb.epoch.TimeIntervals'>,
    trials_table_session_data_2024-09-24 13:13:13 <class 'pynwb.epoch.TimeIntervals'>
  }
  processing: {
    session_spike_times <class 'pynwb.base.ProcessingModule'>
  }
  scratch: {
    psth_normalizers_2024-09-24 12:30:21 <class 'pynwb.core.ScratchData'>,
    psth_session_data_2024-09-24 13:13:13 <class 'pynwb.core.ScratchData'>,
    psth_session_data_concatenated <class 'pynwb.core.ScratchData'>,
    sites_psth_session_data_concatenated <class 'pynwb.core.ScratchData'>,
    timebins_psth_normalizers_2024-09-24 12:30:21 <class 'pynwb.core.ScratchData'>,
    timebins_psth_session_data_2024-09-24 13:13:13 <class 'pynwb.core.ScratchData'>,
    timebins_psth_session_data_concatenated <class 'pynwb.core.ScratchData'>
  }
  session_description: thresholded
  session_id: Apollo_Simulation_thresholded
  session_start_time: 2024-12-23 17:11:58.735689-05:00
  stimulus: {
    stimulus_meta <class 'hdmf.common.table.DynamicTable'>
  }
  timestamps_reference_time: 2024-12-23 17:11:58.735689-05:00

In [3]:
# get PSTH keys
psth_keys = nwbfile.scratch.keys()
# print(f" PSTH keys in psth.scratch: {psth_keys}")
print(f"Dictionary keys in psth.scratch: ")
[print(f"{k}") for k in psth_keys]

Dictionary keys in psth.scratch: 
psth_normalizers_2024-09-24 12:30:21
psth_session_data_2024-09-24 13:13:13
psth_session_data_concatenated
sites_psth_session_data_concatenated
timebins_psth_normalizers_2024-09-24 12:30:21
timebins_psth_session_data_2024-09-24 13:13:13
timebins_psth_session_data_concatenated


[None, None, None, None, None, None, None]

In [4]:
# look for the concatenated (merged) PSTH data that is quality-controlled with the normalizers set
for key in psth_keys:
    if key.startswith("psth_session_data_concatenated"):
        psth_merged = nwbfile.scratch[key]
        print(f"Merged PSTH key: {key}")
        print(f"Merged PSTH shape: {psth_merged.shape}")
        print(f"Merged PSTH description: {psth_merged.description}")
        n_units, n_stimuli, n_reps, n_timebins = psth_merged.shape
        print(f"Merged PSTH dimensions: n_valid_units: {n_units}, n_stimuli: {n_stimuli}, n_reps: {n_reps}, n_timebins: {n_timebins}\n")
    
    if key.startswith("timebins_psth_session_data_concatenated"):
        timebins_object = nwbfile.scratch[key]
        timebins_s = timebins_object[:]
        print(f"Number of PSTH time bins: {timebins_object.shape}")
        print(f"PSTH time bin description: {timebins_object.description}")
        print(f"PSTH timebins (seconds): {timebins_s}")


Merged PSTH key: psth_session_data_concatenated
Merged PSTH shape: (20, 56, 22, 50)
Merged PSTH description: Concatenated PSTH from multiple files
Merged PSTH dimensions: n_valid_units: 20, n_stimuli: 56, n_reps: 22, n_timebins: 50

Number of PSTH time bins: (50,)
PSTH time bin description: Time bins in units of seconds for concatenated PSTH
PSTH timebins (seconds): [-0.2  -0.19 -0.18 -0.17 -0.16 -0.15 -0.14 -0.13 -0.12 -0.11 -0.1  -0.09
 -0.08 -0.07 -0.06 -0.05 -0.04 -0.03 -0.02 -0.01  0.    0.01  0.02  0.03
  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11  0.12  0.13  0.14  0.15
  0.16  0.17  0.18  0.19  0.2   0.21  0.22  0.23  0.24  0.25  0.26  0.27
  0.28  0.29]


In [5]:
# Get stimulus meta info 
stimulus_meta = nwbfile.stimulus["stimulus_meta"].to_dataframe()
stimulus_meta.head() # order of stimuli corresponds to stimulus dimension in the PSTH data


,stimuli_presentation_id,stimuli_filename,stimuli_hash
id,,,
0,1,NormalizerV3_1.png,b6b7852b9451fe5b38ee273b9d395056218b9f6f
1,2,NormalizerV3_2.png,bf2e4a0747042586cc26a9170332ceb4381fa5bc
2,3,NormalizerV3_3.png,2bc8be81044435b2dff4965adc678da23ab05da8
3,4,NormalizerV3_4.png,7506575c9e26f8f49433971113191454b79fd03d
4,5,NormalizerV3_5.png,d66de98203569aed891610c0a3b17e1a30f3e7a6


In [6]:
# You could also access all of the stimulus events for your "session", i.e., non-normalizer recording
stim_events_dict = nwbfile.intervals
stim_events_keys = list(stim_events_dict.keys())
# print(stim_events_keys)

session_stim_events_key = [key for key in stim_events_keys if 'session' in key][0]
print(f"Key associated to your 'session': {session_stim_events_key}")

session_stim_events_df = stim_events_dict[session_stim_events_key].to_dataframe()
session_stim_events_df.head()

Key associated to your 'session': trials_table_session_data_2024-09-24 13:13:13


,start_time,stop_time,stimuli_presentation_time_ms,inter_stimuli_interval_ms,stimulus_presented,fixation_correct,trial_index,stimulus_order_in_trial,stimulus_size_degrees,fixation_window_size_degrees,fixation_point_size_degrees,image_hash,stimulus_filename
id,,,,,,,,,,,,,
0,2.02115,2.12115,100.0,100.0,34,True,0,1,8,3,0.2,57e50ed8a3b88fc603b4d4d713ea24cbc62bc1c6,NormalizerV3_34.png
1,2.22960,2.32960,100.0,100.0,42,True,0,2,8,3,0.2,2c376f5025fb1480ef8460ea9988a0a1dbe6823f,NormalizerV3_42.png
2,2.43800,2.53800,100.0,100.0,31,True,0,3,8,3,0.2,056226cd266e17251b5f4ce03fbe7ed23af47d2d,NormalizerV3_31.png
3,2.64645,2.74645,100.0,100.0,4,True,0,4,8,3,0.2,7506575c9e26f8f49433971113191454b79fd03d,NormalizerV3_4.png
4,2.86185,2.96185,100.0,100.0,27,True,0,5,8,3,0.2,67ccdda7afe6ab00fce51ade33be18287c92a497,NormalizerV3_27.png


In [7]:
# You could also see the quality of each site
analysis_keys = list(nwbfile.analysis.keys())
print(f"Analysis keys: {analysis_keys}")

# looking at the first normalizer set with quality-controlled metrics 
qc_table = nwbfile.analysis[analysis_keys[0]].to_dataframe()
qc_table.head()

Analysis keys: ['normalizer_analysis_0']


,channel_name,p_value,valid_unit,mean_rate,response_latency_ms,half_split_reliability,single_repeat_reliability
id,,,,,,,
0,A-000,0.00,True,65.485307,0.09,0.756725,0.610546
1,A-001,0.16,False,44.354408,0.10,0.344470,0.215752
2,A-002,0.06,False,65.129118,0.08,0.420056,0.272419
3,A-003,0.00,True,93.766696,0.09,0.864655,0.762625
4,A-004,0.00,True,71.772039,0.13,0.651858,0.487674


In [8]:
# spike times
spike_times_dict = nwbfile.processing['session_spike_times'].data_interfaces
spike_times_keys = list(spike_times_dict.keys())
print(f"Number of spike time data_interfaces: {len(spike_times_keys)}")
session_spike_times_key = [key for key in spike_times_keys if 'session' in key][0]
print(session_spike_times_key)
print(spike_times_dict[session_spike_times_key].to_dataframe().head())

Number of spike time data_interfaces: 2
spike_times_session_data_2024-09-24 13:13:13
   unit_name  unit_location_um     probe  \
id                                         
0      A-000  [3600.0, 3200.0]  ProbeABC   
1      A-001  [3200.0, 2800.0]  ProbeABC   
2      A-002  [3600.0, 2800.0]  ProbeABC   
3      A-003  [3600.0, 2400.0]  ProbeABC   
4      A-004  [3600.0, 2000.0]  ProbeABC   

                                          spike_times  \
id                                                      
0   [0.0039, 0.0947, 0.1107, 0.11365, 0.12105, 0.1...   
1   [0.0363, 0.03995, 0.04865, 0.0534, 0.06265, 0....   
2   [0.10855, 0.15575, 0.1764, 0.18125, 0.38635, 0...   
3   [0.0133, 0.05825, 0.16095, 0.2711, 0.3353, 0.3...   
4   [0.0076, 0.04675, 0.0509, 0.05555, 0.07665, 0....   

                                           electrodes  
id                                                     
0      location                                   ...  
1      location                       

In [9]:
#Question: What if i want to reconstruct my own PSTH?
#Answer: instead of the merged PSTHs, use spike times & the corresponding stimulus times from 
# a single session and make your own PSTH 
stim_events_dict = nwbfile.intervals
stim_events_keys = list(stim_events_dict.keys())
print(stim_events_keys)

session_stim_events_key = [key for key in stim_events_keys if 'session' in key][0]
print(session_stim_events_key)

session_stim_events_df = stim_events_dict[session_stim_events_key].to_dataframe()
session_stim_events_df.head()

# spike times
spike_times_dict = nwbfile.processing['session_spike_times'].data_interfaces
spike_times_keys = list(spike_times_dict.keys())
print(f"Number of spike time data_interfaces: {len(spike_times_keys)}")
session_spike_times_key = [key for key in spike_times_keys if 'session' in key][0]
print(session_spike_times_key)
print(spike_times_dict[session_spike_times_key].to_dataframe().head())


['trials_table_normalizers_2024-09-24 12:30:21', 'trials_table_session_data_2024-09-24 13:13:13']
trials_table_session_data_2024-09-24 13:13:13
Number of spike time data_interfaces: 2
spike_times_session_data_2024-09-24 13:13:13
   unit_name  unit_location_um     probe  \
id                                         
0      A-000  [3600.0, 3200.0]  ProbeABC   
1      A-001  [3200.0, 2800.0]  ProbeABC   
2      A-002  [3600.0, 2800.0]  ProbeABC   
3      A-003  [3600.0, 2400.0]  ProbeABC   
4      A-004  [3600.0, 2000.0]  ProbeABC   

                                          spike_times  \
id                                                      
0   [0.0039, 0.0947, 0.1107, 0.11365, 0.12105, 0.1...   
1   [0.0363, 0.03995, 0.04865, 0.0534, 0.06265, 0....   
2   [0.10855, 0.15575, 0.1764, 0.18125, 0.38635, 0...   
3   [0.0133, 0.05825, 0.16095, 0.2711, 0.3353, 0.3...   
4   [0.0076, 0.04675, 0.0509, 0.05555, 0.07665, 0....   

                                           electrodes  
id    